In [ ]:
import cv2
import numpy as np 
import pandas as pd 
import os
import random
import shutil
import matplotlib.pyplot as plt
# image processing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#keras models
from tensorflow.keras import layers ,models,optimizers
from keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.utils import plot_model
#resnet50
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications.imagenet_utils import preprocess_input 
#stop criteria
from tensorflow.keras.callbacks import EarlyStopping
#cnn_visualisation
from keras.preprocessing import image


# moving to working directory

In [ ]:
!cp -r /imgs/train ./


In [ ]:
train_dir = "train/"
valid_dir ="val/"
test_dir  = "test/" 

# defining classes

In [ ]:
activity = {   'c0' : "safe_driving",
                      'c1' : "texting-right",
                      'c2' : "talking_on_the_phone-right",
                      'c3' : "texting-left",
                      'c4' : "talking_on_the_phone-left",
                      'c5' : "operating_the_radio",
                      'c6' : "drinking",
                      'c7' : "reaching_behind",
                      'c8' : "hair-and-makeup",
                      'c9' : "talking_to_passenger"}
activity.keys()

# prepare data
**contact each image to its path and label it**

In [ ]:
for file in os.listdir(train_dir):
    shutil.move(os.path.join(train_dir,file), os.path.join(train_dir,activity[f'{file}']))

In [ ]:
for file in os.listdir(train_dir):
    os.makedirs(valid_dir + '/' + file, exist_ok=True)
    os.makedirs(test_dir + '/' + file, exist_ok=True)    
    train_dir_img = train_dir + file
    file_len = len([sample for sample in os.listdir(train_dir_img)])
    print(file_len)   # no of sampels for each class
    
    for sample in random.sample(os.listdir(train_dir_img) , int(float(0.1) * file_len)):
        shutil.move(train_dir_img + '/' + sample, valid_dir + file)# Moving source to newly created destination 
    for sample in random.sample(os.listdir(train_dir_img) , int(float(0.1) * file_len)):
        shutil.move(train_dir_img + '/' + sample, test_dir + file)

# plot one image from each class

In [ ]:
plt.figure(figsize = (12, 20))
image_count = 1
BASE_URL = '../input/state-farm-distracted-driver-detection/imgs/train/'
for directory in os.listdir(BASE_URL):
    if directory[0] != '.':
        for i, file in enumerate(os.listdir(BASE_URL + directory)):
            if i == 1:
                break
            else:
                fig = plt.subplot(5, 2, image_count)
                image_count += 1
                image =cv2.imread(BASE_URL + directory + '/' + file)
                img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                plt.imshow(img)
                plt.title(activity[directory])

# **RESIZE IMAGES**

In [ ]:
datagen = ImageDataGenerator(rescale=1 / 255.0)
# ImageDataGenerator and usually it is used with rescaling factor 1./255 to rescale the initial values from 0 to 255 to 0 to 1 instead.

In [ ]:
batch_size = 32
train_batches =datagen.flow_from_directory(directory = train_dir,shuffle = True,batch_size = batch_size)
val_batches =datagen.flow_from_directory(directory = valid_dir,shuffle = True, batch_size = batch_size)
test_batches =datagen.flow_from_directory(directory= test_dir,shuffle = False,batch_size = 1)

# Dense model

In [ ]:
network = models.Sequential()
network.add(layers.Flatten(input_shape=(256,256,3)))
network.add(layers.Dense(1024,activation = 'relu',name = 'input'))
network.add(layers.BatchNormalization())
#Batch normalization applies a transformation that maintains the mean 
#output close to 0 and the output standard deviation close to 1.
network.add(layers.Dense(512,activation = 'relu',name = 'l1'))
network.add(layers.BatchNormalization())
network.add(layers.Dense(256,activation = 'relu',name = 'l2'))
network.add(layers.BatchNormalization())
network.add(layers.Dense(10,activation = 'softmax',name = 'output'))

In [ ]:
network.summary()

In [ ]:
plot_model(network,to_file='model.png')

In [ ]:
network.compile(optimizer='Adam',
                loss='categorical_crossentropy',
                metrics=['acc'])

In [ ]:
stop_criteria = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)
dense_model=network.fit(x = train_batches,
          steps_per_epoch=350,
          epochs=15,
          validation_data = val_batches,
          validation_steps= 50
          , callbacks=[stop_criteria]
                       )

In [ ]:
dense_model.history

In [ ]:
plt.plot(dense_model.history['loss'])  # blue line
plt.plot(dense_model.history['val_loss']) #orange line
plt.title('loss')
plt.ylabel('loss')
plt.legend(['train','validation'])

In [ ]:
plt.plot(dense_model.history['acc'])  # blue line
plt.plot(dense_model.history['val_acc']) #orange line
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.legend(['train','validation'])

In [ ]:
dense_scores= network.evaluate(test_batches)
print("Accuracy: %.2f%%" % (dense_scores[1]*100))